In [12]:
import json
from bs4 import BeautifulSoup
import re
import shutil
import os

In [13]:
to_process_directory = rf"../Study questions to process"
processed_directory = rf"../Study Questions processed"

default_raw_db_path = "../study_quiz_raw_questions_db.json"
default_processed_by_quiz_db_path = "../study_quiz_processed_questions_by_quiz_db.json"
default_processed_by_topics_db_path = "../study_quiz_processed_questions_by_topics_db.json"


In [14]:
def reset_database_and_reprocess():
    """
    Resets the database by removing JSON files and moving all processed files
    back to the to-process directory for reprocessing.
    """
    # Delete database files if they exist
    if os.path.exists(default_raw_db_path):
        os.remove(default_raw_db_path)
    
    if os.path.exists(default_processed_by_quiz_db_path):
        os.remove(default_processed_by_quiz_db_path)
    
    # Create empty database files
    with open(default_raw_db_path, 'w', encoding='utf-8') as f:
        json.dump({"quizzes": []}, f)
        print(f"Created empty {default_raw_db_path}")
    
    # with open(default_processed_by_quiz_db_path, 'w', encoding='utf-8') as f:
    #     json.dump([], f)
    #     print(f"Created empty {default_processed_by_quiz_db_path}")
    
    # Ensure to_process_directory exists
    os.makedirs(to_process_directory, exist_ok=True)
    
    # Move files back from processed to to-process
    if os.path.exists(processed_directory):
        moved_count = 0
        for file in os.listdir(processed_directory):
            if file.endswith(".html"):
                source = os.path.join(processed_directory, file)
                target = os.path.join(to_process_directory, file)
                shutil.move(source, target)
                moved_count += 1
    else:
        print(f"Processed directory {processed_directory} does not exist")

# Usage example:
reset_database_and_reprocess()

Created empty ../study_quiz_raw_questions_db.json


In [15]:
if not os.path.exists(default_raw_db_path):
        with open(default_raw_db_path, 'w', encoding='utf-8') as f:
            json.dump({"quizzes": []}, f)


if not os.path.exists(default_processed_by_quiz_db_path):
        with open(default_processed_by_quiz_db_path, 'w', encoding='utf-8') as f:
            json.dump({"quizzes": []}, f)

In [16]:
from pydoc import text


def clean_question(text):
    """Clean up text by removing HTML tags, extra newlines, and normalizing whitespace."""
    if not text:
        return ""
    
    # Replace HTML entities
    # text = text.replace('<sup>;', '^')
    # text = text.replace('<br>;', ' ')
    text = text.replace('&nbsp;', ' ')
    # Remove HTML tags
    # text = re.sub(r'<[^>]+>', '', text)

    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    
    # # Replace newlines followed by spaces
    text = re.sub(r'\n\s+', ' ', text)
    
    # # Replace any remaining newlines with spaces
    # text = re.sub(r'\n+', ' ', text)
    
    # Normalize whitespace (multiple spaces to single space)
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def clean_comment(text):
    """Clean up text by removing HTML tags, extra newlines, and normalizing whitespace."""
    if not text:
        return ""
    
    # Replace HTML entities
    # text = text.replace('<sup>;', '^')
    # text = text.replace('<br>;', ' ')
    text = text.replace('&nbsp;', ' ')
    # Remove HTML tags
    # text = re.sub(r'<[^>]+>', '', text)

    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    
    # # Replace newlines followed by spaces
    # text = re.sub(r'\n\s+', ' ', text)
    
    # # Replace any remaining newlines with spaces
    # text = re.sub(r'\n+', ' ', text)
    
    # Normalize whitespace (multiple spaces to single space)
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def extract_quiz_to_json(html_path):
    with open(html_path, encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    base_url=rf"https://canvas.umn.edu"

    # Get quiz title from <title> tag
    title_tag = soup.find("title")
    quiz_title = clean_question(title_tag.text) if title_tag else ""
    quiz_title = quiz_title.replace("Joshua Cheng's Quiz History: ", "").strip()
    quiz_title = quiz_title.replace(": BIOL 1009 (070-082) General Biology (Fall 2025)", "").strip()
    quiz_title = quiz_title.replace(" (course resources and policies)", "").strip()
    
    pattern = r'\s*\(due\s+\w+,\s+\w+\.\s+\d+\s+at\s+[\d:]+\s+[AP]M\)'
    
    # Replace the pattern with empty string
    quiz_title = re.sub(pattern, '', quiz_title)

    questions = []

    def extract_image_url(html_string):
        # Match image URLs in src attributes
        html_string = str(html_string).strip()
        if "img" not in html_string:
            return ""
        
        # Try with regular quotes first (which is what your examples have)
        img_pattern = r'<img[^>]*src="([^"]+)"'
        
        # Find all matches
        matches = re.findall(img_pattern, html_string)
        
        # If no matches, try with escaped quotes
        if not matches:
            img_pattern = r'<img[^>]*src=\\"([^"\\]+)\\"'
            matches = re.findall(img_pattern, html_string)

        if matches:
            img_url = matches[0]
            # If it's a relative URL, add the base domain
            if img_url.startswith('/'):
                img_url = base_url + img_url
            return img_url
        else:
            return ""
    
    for q_holder in soup.find_all("div", class_="question_holder"):
        question = {}
        question_image_url = ""

        # Try to get question text - first from original_question_text, then from question_text
        q_text = q_holder.find("div", class_="original_question_text")
        if q_text and q_text.text.strip():
            question_image_url = extract_image_url(q_text.text)
            question_text = clean_question(q_text.text)
        else:
            # Try getting question from question_text which may contain images
            q_content = q_holder.find("div", class_="question_text user_content enhanced")
            if q_content:
                question_image_url = extract_image_url(q_content.text)
                # For questions with images, keep the HTML content
                question_text = str(q_content)
            else:
                # Fall back to display_question
                q_display = q_holder.find("div", class_="display_question")
                if q_display:
                    # Sometimes question text is in aria-label or inside the div
                    question_image_url = extract_image_url(q_display.text)
                    question_text = clean_question(q_display.get("aria-label", ""))
                    if not question_text:
                        question_text = clean_question(q_display.text)
                else:
                    question_text = ""
        
        question["question"] = question_text
        question["question_image_url"] = question_image_url

        answers = []
        correct_answer = None
        for ans_div in q_holder.find_all("div", class_="answer"):
            ans = {}
            ans_text_div = ans_div.find("div", class_="answer_text")
            ans_text = clean_question(ans_text_div.text) if ans_text_div else ""
            ans["text"] = ans_text

            # Detect correct answer by class or presence of correct arrow
            is_correct = (
                "correct_answer" in ans_div.get("class", []) or
                "correct" in ans_div.get("class", []) or
                ans_div.find("span", class_="answer_arrow correct")
            )
            ans["is_correct"] = bool(is_correct)
            if ans["is_correct"]:
                correct_answer = ans_text

            # Get comment if present
            comment_div = ans_div.find("div", class_="quiz_comment")
            if comment_div:
                comment_text = clean_question(comment_div.text)
                ans["comment"] = comment_text
            else:
                ans["comment"] = None

            answers.append(ans)

        question["answers"] = answers
        question["correct_answer"] = correct_answer

        # Get any general comments for the question (outside answer divs)
        quiz_comment_divs = q_holder.find_all("div", class_="quiz_comment")
        
        # Initialize comment fields
        # question["correct_comments"] = None
        question["neutral_comments"] = None
        
        if quiz_comment_divs:
            all_comment_text = []
            
            for comment_div in quiz_comment_divs:
                if comment_div.text.strip():
                    all_comment_text.append(clean_comment(comment_div.text))
                    # all_comment_text.append(comment_div.text)
                    question["all_comments"] = comment_div.text
                
                neutral_comments_p = comment_div.find("p", class_="neutral_comments")
                
                if neutral_comments_p and not question["neutral_comments"]:
                    question["neutral_comments"] = clean_comment(neutral_comments_p.text)
                    # question["neutral_comments"] = neutral_comments_p.text

        questions.append(question)

    # Save to JSON
    output = {
        "quiz_title": quiz_title,
        "questions": questions
    }
    return output

def moved_processed_file(file):
    shutil.move(rf"{os.path.join(to_process_directory, file)}", rf"{os.path.join(processed_directory, file)}")

def get_processed_quizes(db_path=default_raw_db_path, debug=0):
    if not os.path.exists(db_path):
        with open(db_path, 'w', encoding='utf-8') as f:
            json.dump({"quizzes": []}, f)

    with open(db_path, 'r', encoding='utf-8') as f:
        db = json.load(f)
    quizzes = [quiz["quiz_title"] for quiz in db.get("quizzes", [])]
    if debug:
        print(f"Processed quizzes: {quizzes}")
    return quizzes

def insert_json_quiz_into_db(quiz_data, db_path=default_raw_db_path):
    if not os.path.exists(db_path):
        with open(db_path, 'w', encoding='utf-8') as f:
            json.dump({"quizzes": []}, f)

    # Read current database
    with open(db_path, 'r', encoding='utf-8') as f:
        db = json.load(f)

    # Check for existing titles and increment suffix if needed
    base_title = quiz_data["quiz_title"]
    existing_titles = [quiz["quiz_title"] for quiz in db.get("quizzes", [])]
    new_title = base_title
    suffix = 2
    while new_title in existing_titles:
        new_title = f"{base_title}_{suffix}"
        suffix += 1
    quiz_data["quiz_title"] = new_title

    # Add new quiz to database
    db["quizzes"].append(quiz_data)

    # Write updated database
    with open(db_path, 'w', encoding='utf-8') as f:
        json.dump(db, f, indent=2, ensure_ascii=False)

    # print(f"Added quiz '{new_title}' to database.")
    return True

# Now update your file processing function to use this
def process_file(file_name):
    file_path = os.path.join(to_process_directory, file_name)
    quiz_data = extract_quiz_to_json(file_path)
    
    if insert_json_quiz_into_db(quiz_data):
        os.makedirs(processed_directory, exist_ok=True)
        
        shutil.move(file_path, os.path.join(processed_directory, file_name))
    else:
        print(f"Skipped {file_name}, quiz already in database.")

def process_directory(folder_path=to_process_directory):
    for file in os.listdir(folder_path):
        if file.endswith(".html"):
            process_file(file)

In [17]:
get_processed_quizes(debug=1)

Processed quizzes: []


[]

In [18]:
process_directory()

In [19]:
def get_indexed_quiz_titles(quiz_title, db_path=default_processed_by_quiz_db_path):
    # print("\n", quiz_title, end="")
    try:
        with open(db_path, "r", encoding="utf-8") as f:
            refined_db = json.load(f)

        for idx, quiz in enumerate(refined_db):
            if quiz["quiz_title"] == quiz_title:
                # print(f" == {quiz['quiz_title']}")
                return idx
            # else:
            #     print(f" != {quiz['quiz_title']}", end="")
        return -1
    except FileNotFoundError:
        return -1

def refine_quiz_db_by_quiz(raw_db):
    # Track unique questions and assign IDs
    question_bank = {}
    question_id_counter = 1
    refined_quizzes = []

    pattern = r'_\d+$'
    
    process_last = []
    for quiz in raw_db["quizzes"]:
        if quiz["quiz_title"].startswith("Quiz") or quiz["quiz_title"].startswith("SQ midterm"):
            process_last.append(quiz["quiz_title"])

    sorted_quizzes = sorted(raw_db["quizzes"], key=lambda quiz: (
        quiz["quiz_title"] in process_last,  # False (0) comes before True (1)
        quiz["quiz_title"]  # Secondary sort by title for consistent ordering
    ))

    print(sorted_quizzes)

    for quiz in sorted_quizzes:
        base_quiz_title = re.sub(pattern, '', quiz["quiz_title"])

        refined_quiz = {
            "quiz_title": base_quiz_title,
            "questions": []
        }
        unique_count = 0

        for q in quiz["questions"]:
            q_text = q["question"].strip()
            q_image_url = q.get("question_image_url", "")
            unique_key = (q_text, q_image_url)

            # Check if question is unique
            if unique_key not in question_bank:
                question_bank[unique_key] = question_id_counter
                unique = True
                q_id = question_id_counter
                question_id_counter += 1
                unique_count += 1
            else:
                continue

            # Build options list
            options = []
            for ans in q["answers"]:
                options.append({
                    "text": ans["text"],
                    "is_correct": ans["is_correct"],
                    "comment": ans.get("comment")
                })

            refined_quiz["questions"].append({
                "question": q_text,
                "question_image_url": q_image_url,
                "unique": unique,
                "id": q_id,
                "options": options,
                "neutral_comments": q.get("neutral_comments")
            })

        # Check if this base title already exists in our refined_quizzes list
        existing_quiz_index = -1
        for idx, existing_quiz in enumerate(refined_quizzes):
            if existing_quiz["quiz_title"] == base_quiz_title:
                existing_quiz_index = idx
                break
        
        if existing_quiz_index == -1:
            # This is the first time we see this base quiz title
            refined_quiz["unique_questions"] = unique_count
            refined_quizzes.append(refined_quiz)
        else:
            # This quiz already exists, merge questions
            # print(f"Merging questions from '{quiz['quiz_title']}' into '{base_quiz_title}'")
            refined_quizzes[existing_quiz_index]["questions"].extend(refined_quiz["questions"])
            refined_quizzes[existing_quiz_index]["unique_questions"] += unique_count
    
    return refined_quizzes
# Usage:
with open(default_raw_db_path, encoding="utf-8") as f:
    raw_db = json.load(f)

# refined_db = refine_quiz_db_by_quiz(raw_db)

# with open(default_processed_by_quiz_db_path, "w", encoding="utf-8") as f:
#     json.dump(refined_db, f, indent=2, ensure_ascii=False)

In [20]:
def refine_quiz_db_by_topic(raw_db):
    # Track unique questions and assign IDs
    question_bank = {}
    question_id_counter = 1
    refined_quizzes = []

    base_quiz_title_pattern = r'((SQ topics?\s\d+(-\d+)?(\s(and)\s\d+)?)|(Quiz \d+)|(SQ midterm \d+))'

    process_second_pattern = r"SQ topics?\s\d+-\d+"
    
    process_third = []
    process_second = []
    for quiz in raw_db["quizzes"]:
        if quiz["quiz_title"].startswith("Quiz") or quiz["quiz_title"].startswith("SQ midterm"):
            process_third.append(quiz["quiz_title"])
        elif re.match(process_second_pattern, quiz["quiz_title"]):
            # print(f"Processing second pattern quiz: {re.match(process_second_pattern, quiz['quiz_title']).group(0)}")
            title2 = re.match(process_second_pattern, quiz['quiz_title']).group(0)
            # print(f"Processing second pattern quiz: {title2}")
            process_second.append(quiz["quiz_title"])

    sorted_quizzes = sorted(raw_db["quizzes"], key=lambda quiz: (
        3 if quiz["quiz_title"] in process_third else 2 if quiz["quiz_title"] in process_second else 1,
        # quiz["quiz_title"]  # Secondary sort by title for consistent ordering
    ))

    # print(sorted_quizzes)

    for quiz in sorted_quizzes:
        topic_title = re.match(base_quiz_title_pattern, quiz['quiz_title'])
        if topic_title:
            topic_title = topic_title.group(1)

        if quiz['quiz_title'] == "" and len(quiz['questions']) == 0:
            continue
        print(f"base processing quiz: {quiz['quiz_title']} --> {topic_title if topic_title else quiz['quiz_title']}")

        if not topic_title:
            topic_title = quiz['quiz_title']

        base_quiz_title = topic_title

        # print(f"quiz title: '{quiz['quiz_title']}' -> base title: '{base_quiz_title}'")

        refined_quiz = {
            "quiz_title": base_quiz_title,
            "questions": []
        }
        unique_count = 0

        for q in quiz["questions"]:
            q_text = q["question"].strip()
            q_image_url = q.get("question_image_url", "")
            unique_key = (q_text, q_image_url)

            # Check if question is unique
            if unique_key not in question_bank:
                question_bank[unique_key] = question_id_counter
                unique = True
                q_id = question_id_counter
                question_id_counter += 1
                unique_count += 1
            else:
                continue

            # Build options list
            options = []
            for ans in q["answers"]:
                options.append({
                    "text": ans["text"],
                    "is_correct": ans["is_correct"],
                    "comment": ans.get("comment")
                })

            refined_quiz["questions"].append({
                "question": q_text,
                "question_image_url": q_image_url,
                "unique": unique,
                "id": q_id,
                "options": options,
                "neutral_comments": q.get("neutral_comments")
            })

        # Check if this base title already exists in our refined_quizzes list
        existing_quiz_index = -1
        for idx, existing_quiz in enumerate(refined_quizzes):
            if existing_quiz["quiz_title"] == base_quiz_title:
                existing_quiz_index = idx
                break
        
        if existing_quiz_index == -1:
            # This is the first time we see this base quiz title
            refined_quiz["unique_questions"] = unique_count
            refined_quizzes.append(refined_quiz)
        else:
            # This quiz already exists, merge questions
            # print(f"Merging questions from '{quiz['quiz_title']}' into '{base_quiz_title}'")
            refined_quizzes[existing_quiz_index]["questions"].extend(refined_quiz["questions"])
            refined_quizzes[existing_quiz_index]["unique_questions"] += unique_count
    
    return refined_quizzes

# Usage:
with open(default_raw_db_path, encoding="utf-8") as f:
    raw_db = json.load(f)

refined_db = refine_quiz_db_by_topic(raw_db)

with open(default_processed_by_topics_db_path, "w", encoding="utf-8") as f:
    json.dump(refined_db, f, indent=2, ensure_ascii=False)

base processing quiz: SQ topic 1a level 1a --> SQ topic 1
base processing quiz: SQ topic 1a level 1b --> SQ topic 1
base processing quiz: SQ topic 1a level 1c --> SQ topic 1
base processing quiz: SQ topic 1a level 1d --> SQ topic 1
base processing quiz: SQ topic 1a level 2 --> SQ topic 1
base processing quiz: SQ topic 1b level 1a (Chapter 1) --> SQ topic 1
base processing quiz: SQ topic 1b level 1b (Chapter 1) --> SQ topic 1
base processing quiz: SQ topic 1b level 1c (Chapter 1) --> SQ topic 1
base processing quiz: SQ topic 1b level 1d (Chapter 1) --> SQ topic 1
base processing quiz: SQ topic 1b level 2 (Chapter 1) --> SQ topic 1
base processing quiz: SQ topic 1b level 2 (Chapter 1)_2 --> SQ topic 1
base processing quiz: SQ topic 1b level 2 (Chapter 1)_3 --> SQ topic 1
base processing quiz: SQ topic 1b level 2 (Chapter 1)_4 --> SQ topic 1
base processing quiz: SQ topic 1b level 2 (Chapter 1)_5 --> SQ topic 1
base processing quiz: SQ topic 1b level 2 (Chapter 1)_6 --> SQ topic 1
base pr

In [21]:
def print_quiz_titles_and_unique_questions_table(db_path=default_processed_by_quiz_db_path):
    with open(db_path, encoding="utf-8") as f:
        quizzes = json.load(f)
    col_1_len = 60
    col_2_len = 15
    print(f"{'Quiz Title':<{col_1_len}} | {'Unique Questions':<{col_2_len}}")
    print("-" * (col_1_len + col_2_len))
    unique_count = 0
    for quiz in quizzes:
        title = quiz.get('quiz_title', '(no title)')
        # if "SQ topic 2 level" not in title:
        #     continue
        unique_q = quiz.get('unique_questions', 0)
        unique_count += unique_q
        print(f"{title:<{col_1_len}} | {unique_q:<{col_2_len}}")
    print(f"{'Total':<{col_1_len}} | {unique_count:<{col_2_len}}")

print_quiz_titles_and_unique_questions_table(db_path=default_processed_by_quiz_db_path)

Quiz Title                                                   | Unique Questions
---------------------------------------------------------------------------


AttributeError: 'str' object has no attribute 'get'

In [ ]:
print_quiz_titles_and_unique_questions_table(db_path=default_processed_by_topics_db_path)


Quiz Title                                                   | Unique Questions
---------------------------------------------------------------------------
SQ topic 1                                                   | 51             
SQ topic 2                                                   | 72             
SQ topic 3                                                   | 43             
SQ topic 4                                                   | 59             
SQ topic 7                                                   | 54             
SQ topics 5 and 6                                            | 63             
SQ topics 1-4                                                | 9              
SQ topics 5-7                                                | 16             
Quiz 1                                                       | 2              
Quiz 2                                                       | 4              
Quiz 3                                                